## Setup and data read

In [ ]:
# Imports and parameter setting
from pyspark.sql import SparkSession, Row
from pyspark.sql import functions as Funcs

from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, Imputer
from sklearn.linear_model import ElasticNet, ElasticNetCV
from sklearn.metrics import mean_squared_error
from sklearn import ensemble

import pandas as pd
import numpy as np
import cPickle as pickle

pd.set_option("display.max_columns", 100)

inputPath = '/home/jovyan/work/data/autot4.7.csv'

# Create a spark session
session = SparkSession \
    .builder \
    .appName("Car data") \
    .config('spark.driver.memory', '5G') \
    .config('spark.executor.memory', '5G') \
    .getOrCreate()
    


In [ ]:
# Read input data into a spark data frame
# remove . from column names
inDf = session.read \
    .format("org.apache.spark.csv") \
    .option("header", "true") \
    .option("delimiter", ";") \
    .option("mode", "DROPMALFORMED") \
    .csv(inputPath)
    
newColnames = [col.replace('.','_',5) for col in inDf.columns]
inDf = inDf.toDF(*newColnames)

In [ ]:
#Select a subset of columns and set their types 
carsDf = inDf.select(
    'ajoneuvoluokka',
    'ajoneuvonkaytto',
    'ajoneuvoryhma',
    'korityyppi',
    'ohjaamotyyppi',
    'kayttovoima',
    'istumapaikkojenLkm',
    'sylintereidenLkm',
    'vaihteisto',
    'alue',
    'kunta',
    'merkki',
    'malli',
    'merkki_l_malli',
    'kayttoonotto_pvm_imputoitu',
    inDf['omamassa'].cast("int"),
    inDf['iskutilavuus'].cast("int"),
    inDf['suurinNettoteho'].cast("int"),
    inDf['matkamittarilukema'].cast("int"),
    inDf['kayttoonottoVuosi'].cast("int"),
    inDf['ensirekVuosi'].cast("int"),
    inDf['ensirekisterointipvm'].cast("timestamp"),
    inDf['kayttoonottopvm'].cast("timestamp"),
    inDf['max_date'].cast("timestamp"),
    inDf['kayttoonotto'].cast("timestamp")
)
# List of variables by types strings are called 'factors'
factorVars = [
    'ajoneuvoluokka',
    'ajoneuvonkaytto',
    'ajoneuvoryhma',
    'korityyppi',
    'ohjaamotyyppi',
    'kayttovoima',
    'istumapaikkojenLkm',
    'sylintereidenLkm',
    'vaihteisto',
    'alue',
    'kunta',
    'merkki',
    'malli',
    'merkki_l_malli',
    'kayttoonotto_pvm_imputoitu'
]
numericVars = [
    'omamassa',
    'iskutilavuus',
    'suurinNettoteho',
    'matkamittarilukema',
    'kayttoonottoVuosi',
    'ensirekVuosi'
]
dateVars = [
    'ensirekisterointipvm',
    'kayttoonottopvm',
    'max_date',
    'kayttoonotto'
]


In [ ]:
# Calculate new columns from original
carsDf = carsDf.withColumn(
    'usageDays', 
    (carsDf['max_date'].cast('long')-carsDf['kayttoonottopvm'].cast('long'))/(24.0 * 3600.0)
)
numericVars.append('usageDays')

carsDf = carsDf.withColumn(
    'mileagePerDay', 
    carsDf['matkamittarilukema'].cast('float')/carsDf['usageDays']
)
numericVars.append('mileagePerDay')

# Truncate values to sensible values
carsDf = carsDf.withColumn(
    'mileagePerDay', 
    Funcs.when(carsDf['mileagePerDay']>200, 200).otherwise(carsDf['mileagePerDay'])
)
# Or filter out unsensible values (car mass)
carsDf = carsDf.filter(carsDf.omamassa.between(1, 4000))

carsDf = carsDf.filter(carsDf.matkamittarilukema.between(1e4, 1e6))

#Only cars that are still in use and data has been collected
carsDf = carsDf.filter(carsDf.kayttoonottoVuosi.between(1990, 2011))

# Imputation gives strange results
carsDf = carsDf.filter(carsDf.kayttoonotto_pvm_imputoitu == 0)

## Preprocess for modeling

In [ ]:
# split data into training (20%), test (10%) and rest (70%) sets
# numbers are chosen here for convenience, 20% of this set is enought to fit model
splits = carsDf.randomSplit([0.1, 0.1, 0.8], 220274)

In [ ]:
# Get modelling data to pandas data frame
modelDf = splits[0].toPandas()

In [ ]:
# Get test data into pandas DF
# testDf = splits[1].toPandas()

In [ ]:
# Some more filtering
modelDf = modelDf[
    (modelDf.ajoneuvonkaytto == 'Yksityinen') &
    (modelDf.ajoneuvoryhma.isin(['NA', 'Matkailuauto', 'Maastohenkilöauto']))
]         


In [ ]:
# numeric and factor type variables are treated differently

# first pick numerical variables into training data
trainDf = modelDf[numericVars]

# remove target variable into separate vector
trainDf.pop('mileagePerDay')
target = trainDf.pop('matkamittarilukema')
target = np.log(target)


# numerical data has missing values, replace missing with average of that variable
# Good idea: make additional variable for denoting that the value was missing
imputer = Imputer()
XImputed = imputer.fit_transform(trainDf)

# scale numerical variables to zero mean and unit variance
scaler = StandardScaler()
XScaled = scaler.fit_transform(XImputed)

# put model fitting data into pandas data frame
X = pd.DataFrame(XScaled, columns=trainDf.columns, index=trainDf.index)
X.head()

In [ ]:
target.hist(bins=100)

In [ ]:
# factor variables are included through dummy variable encoding
vars = ['ajoneuvoryhma', 'kayttovoima', 'vaihteisto', 'istumapaikkojenLkm', 'ohjaamotyyppi', 'korityyppi']
for c in vars:
    tmp = pd.get_dummies(modelDf[c], prefix=c)
    # add dummy variables to fitting data
    X[tmp.columns] = tmp

print(X.shape)
X.head()    

## Model fitting


### Linear model
Fit a linear model to the data. Fitting done with elastic-net algorithm
http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html#sklearn.linear_model.ElasticNet.fit

In [ ]:
# for now, just using the default parameters (usually not enough)
enet = ElasticNet()
enet.fit(X, target)

# print results
pd.Series(enet.coef_, index=X.columns)

In [ ]:
# Plot the prediction vs. true values
plotDf = pd.DataFrame({'prediction': enet.predict(X), 'true_value': target})
plotDf['residual'] = plotDf.true_value-plotDf.prediction
zz = np.array([0, plotDf.prediction.max()])
#fig, axes = plt.subplots()
plotDf.plot.scatter(x='prediction', y='true_value')
plt.plot(zz, zz, 'r-')
plt.show()

In [ ]:
sns.distplot(plotDf.residual, bins=100, kde=False)
plt.xlim(-4e5,4e5)

In [ ]:
l1s = (1-np.logspace(0,-2,num=5))
enetCV = ElasticNetCV(l1_ratio=l1s, alphas=np.logspace(-8,-3,num=5), max_iter=5000)
enetCV.fit(X, target)

# print results
print(enetCV.alphas_)
print(enetCV.alpha_)
print(l1s)
print(enetCV.l1_ratio_)
pd.Series(enetCV.coef_, index=X.columns)

In [ ]:
aa = pd.DataFrame(enetCV.mse_path_.mean(axis=2), index=l1s, columns=enetCV.alphas_)
sns.heatmap(aa)

In [ ]:
# Plot the prediction vs. true values
plotDf = pd.DataFrame({'prediction': enetCV.predict(X), 'true_value': target})
plotDf['residual'] = plotDf.true_value-plotDf.prediction
zz = np.array([plotDf.prediction.min(), plotDf.prediction.max()])
#fig, axes = plt.subplots()
plotDf.plot.scatter(x='prediction', y='true_value', alpha=.02)
plt.plot(zz, zz, 'r-')
plt.show()
sns.distplot(plotDf.residual, bins=100, kde=False)
plt.show()

### Gradient boosting
http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html#sklearn.ensemble.GradientBoostingRegressor

In [ ]:
gbr = ensemble.GradientBoostingRegressor(
    n_estimators=400,
    max_depth=3,
    min_samples_split=5,
    learning_rate=0.02,
    loss='lad',
    max_features='sqrt',
    verbose=1
)
gbr.fit(X, target)

In [ ]:
# Plot the prediction vs. true values
plotDf = pd.DataFrame({'prediction': gbr.predict(X), 'true_value': target})
plotDf['residual'] = plotDf.true_value-plotDf.prediction
zz = np.array([plotDf.prediction.min(), plotDf.prediction.max()])
#fig, axes = plt.subplots()
plotDf.plot.scatter(x='prediction', y='true_value', alpha=.1)
plt.plot(zz, zz, 'r-')
plt.show()
sns.distplot(plotDf.residual, bins=100, kde=True)
plt.show()

In [ ]:
feature_importance = gbr.feature_importances_
# make importances relative to max importance
feature_importance = 100.0 * (feature_importance / feature_importance.max())
sorted_idx = np.argsort(feature_importance)

In [ ]:
featDf=pd.Series(data=feature_importance, index=X.columns)
featDf.plot(kind='barh', figsize=(7, 15))

### Save the fitted model and other relevant data

In [ ]:
pickle.dump(
    {
        'model': gbr, 
        'scaler': scaler, 
        'imputer': imputer, 
        'trainColumns': X.columns,
        'factorVars': factorVars,
        'numericVars': numericVars,
        'dateVars': dateVars
    }, 
    open('enet.pyobj','wb')
)